In [1]:
import numpy as np
from pathlib import Path
# data manager and analysis
import numan as nu
import vodex as vx

# Project structure:

Provide the project folder with the "processed" folder created in the previous notebook.

As you keep going with the analysis, the folder will have the following structure:


```
...........................................................
....................... DONE in 01 ........................
...........................................................
processed                                               ...
│   experiment.json <-----------------------------------... the file that contains everything about the experiment, you are creating it once and will be reusing ever after
│   experiment_dff.json <-------------------------------... everything about the experiment, but loads from the dff movie, not from the raw data                   ...
└───dff_movie  <----------------------------------------...the dff movie :)
│   │   dff_movie_0000.tif                              ...
│   │   dff_movie_0001.tif                              ...
│   │   ...                                             ...
│..........................................................
│...................... DONE in 02 ........................
│..........................................................
│                                                       ...
└───tscore_volumes  <-----------------------------------... t-score tif files
│   │   tscore_SvB.tif <--------------------------------... t-score Stimuli vs Blank
│   │                                                   ...
│..........................................................
│...................... DONE : MANUAL .....................
│..........................................................
└───spots                                               ...
│   └───imaris  <---------------------------- ATTENTION ... You need to put stuff generated by imaris into this folder!!!                                         ...
│       │   └───tscore_SvB_Statistics                   ...
│       │       │     tscore_SvB_Position.csv           ...
│       │       │     tscore_SvB_Diameter.csv           ...
│       │       │     ...                               ...
│..........................................................
│................... DONE in 03 & 04 ......................
│..........................................................
│   │                                                   ...
│   └───signals  <--------------------------------------... json files with the extracted signals, also will have the group info after you added it                 ...
│       │   spots_SvB_max.json                          ...
│       │                                               ...
│..........................................................
│...................... DONE in 05 ........................
│..........................................................
│   │                                                   ...
│   └───reports  <------------------------------------- ... tiffs and pdf with the cells significant in any pairwise comparison
│       └───all_significant  <------------------------- ... tiffs and pdf with all significant in any way cells
│           │   └───signals  <------------------------- ... pdfs with signals
│           │       │     ...                           ...
│..........................................................
│..........................................................
│..........................................................
│           │       │
│           │   └───images <--------------------------- tif masks
│           │       │     ...
│..........................................................
│...................... DONE in 06 ........................
│..........................................................
│       │                                               ...
│       └───groupped  <-------------------------------- ... tiffs and pdf where the cells are groupped based on signal shape .. or anything else you want
│           │   readme.txt  <-------------------------- ... ATTENTION : you need to describe the groups
│           │   └───signals  <------------------------- ... pdfs with signals
│           │       │     ...                           ...
│           │   └───images  <-------------------------- ... tif masks
│           │       │     ...                           ...
|++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++ WILL BE DONE in this notebook +++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│       │                                               +++
│       └───covariates  <------------------------------ +++ tiffs and pdf where the cells are groupped based on covariates
│           │   └───signals  <------------------------- +++ pdfs with signals split by covariates
│           │       │     ...                           +++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

```

# Set project folder

The processed/spots/signals should already exist and have the extracted signals saved in there.

In [2]:
project_folder = "D:/Code/repos/numan/data/test/ld00_casper/"
project = nu.Project(project_folder)

project.check_exists("processed/spots/reports/all_significant/signals")

project.activate("processed")

In [3]:
# get the folder ready
experiment = vx.Experiment.load('experiment_raw.db')

## Create a reports folder

In [4]:
project.create("processed/spots/reports/covariates/signals")

## Add time annotation for covariates to the experiment

In [5]:
# you will need this to create the new annotations
frames_per_volume = experiment.db.get_fpv()
n_frames = experiment.db.get_n_frames()

In [6]:
# equivalent convex hull vs equivalent inter-distance
spread = vx.Labels("spread",
                   ["ch","id"],
                   group_info = "characterise dots spread: equivalent convex hull or equivalent inter-distance",
                   state_info={"ch": "equivalent convex hull",
                               "id": "equivalent inter-distance"})
spread_cycle = vx.Cycle(   # note how you can use the name of the stimuli as an attribute!
                        # for this reason it's important the names don't have a space or any other special characters in them
                        [
                          spread.ch,
                          spread.id
                        ],
                        # the duration should be in frames!
                        # here it's first listed in volumes and then multiplied by frames per volume
                        np.array(
                            [
                             94, # full dot cycle of equivalent convex hull
                             94  # full dot cycle of equivalent inter-distance
                            ]
                        )*frames_per_volume
                    )
spread_annotation = vx.Annotation.from_cycle(n_frames, spread, spread_cycle,
                                          info = "characterise dots spread")
spread_annotation

Annotation type: spread
characterise dots spread
Total frames : 44086

In [7]:
# equivalent convex hull vs equivalent inter-distance
shape = vx.Labels("shape",
                   ["cr","ta","tp","b"],
                   group_info = "characterise dots shape",
                   state_info={"b": "blank, shape doesn't apply",
                               "cr": "constant radius",
                               "ta": "equivalent total area",
                               "tp": "equivalent total perimeter"})
shape_cycle = vx.Cycle(   # note how you can use the name of the stimuli as an attribute!
                        # for this reason it's important the names don't have a space or any other special characters in them
                        [
                          shape.b, shape.cr,  # blank , then CR (2)
                          shape.b, shape.cr,  # blank , then CR (5)
                          shape.b, shape.cr,  # blank , then CR (1)
                          shape.b, shape.ta,  # blank , then TA (2)
                          shape.b, shape.ta,  # blank , then TA (5)
                          shape.b, shape.cr,  # blank , then CR (3)
                          shape.b, shape.ta,  # blank , then TA (1)
                          shape.b, shape.ta,  # blank , then TA (3)
                          shape.b, shape.tp,  # blank , then TP (1)
                          shape.b, shape.tp,  # blank , then TP (3)
                          shape.b, shape.tp,  # blank , then TP (5)
                          shape.b, shape.tp,  # blank , then TP (2)
                          shape.b            # blank
                        ],
                        # the duration should be in frames!
                        # here it's first listed in volumes and then multiplied by frames per volume
                        np.array(
                            [
                             3,1,   # blank for 3.0 volumes, then CR (2) for 1 volume;
                             5,1,   # blank for 5.0 volumes, then CR (5) for 1 volume;
                             6,1,   # blank for 6.0 volumes, then CR (1) for 1 volume;
                             9,1,   # blank for 9.0 volumes, then TA (2) for 1 volume;
                             5,1,   # blank for 5.0 volumes, then TA (5) for 1 volume;
                             6,1,   # blank for 6.0 volumes, then CR (3) for 1 volume;
                             9,1,   # blank for 9.0 volumes, then TA (1) for 1 volume;
                             5,1,   # blank for 5.0 volumes, then TA (3) for 1 volume;
                             9,1,   # blank for 9.0 volumes, then TP (1) for 1 volume;
                             7,1,   # blank for 7.0 volumes, then TP (3) for 1 volume;
                             9,1,   # blank for 9.0 volumes, then TP (5) for 1 volume;
                             6,1,   # blank for 6.0 volumes, then TP (2) for 1 volume;
                             3      # blank for 3.0 volumes
                            ]
                        )*frames_per_volume
                    )
shape_annotation = vx.Annotation.from_cycle(n_frames, shape, shape_cycle,
                                          info = "characterise dots shape")
shape_annotation

Annotation type: shape
characterise dots shape
Total frames : 44086

In [8]:
experiment.add_annotations([spread_annotation, shape_annotation])
experiment.save("experiment_w_covariates_raw.db")

Copied 1980 of 1980 pages...


## Make PSH plots for covariates:
## 1. CR : Constant Radius
### Only stimulus
Specify the group for which you want the plots ( by default set to "sigAny1v2v3v5vB", which are all the cells found )

In [9]:
group_to_plot = "sigAny1v2v3v5vB"
# and start the report maker
report = nu.Reports(Path(project.main_folder,"processed"), experiment)

In [10]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("shape", "cr")],
                                         [("number", "d2"), ("shape", "cr")],
                                         [("number", "d3"), ("shape", "cr")],
                                         [("number", "d5"), ("shape", "cr")]
                                         ],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_CR',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[

In [11]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                             [("number", "d1"), ("shape", "cr")],
                                             [("number", "d2"), ("shape", "cr")],
                                             [("number", "d3"), ("shape", "cr")],
                                             [("number", "d5"), ("shape", "cr")]
                                             ],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_CR_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[

### Stimulus and 2 blanks before and 4 after

In [12]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                             [("number", "d1"), ("shape", "cr")],
                                             [("number", "d2"), ("shape", "cr")],
                                             [("number", "d3"), ("shape", "cr")],
                                             [("number", "d5"), ("shape", "cr")]
                                             ],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_CR',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 

In [13]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                             [("number", "d1"), ("shape", "cr")],
                                             [("number", "d2"), ("shape", "cr")],
                                             [("number", "d3"), ("shape", "cr")],
                                             [("number", "d5"), ("shape", "cr")]
                                             ],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_CR_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 

## 2. TA : equivalent total area
### Only stimulus

In [14]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("shape", "ta")],
                                         [("number", "d2"), ("shape", "ta")],
                                         [("number", "d3"), ("shape", "ta")],
                                         [("number", "d5"), ("shape", "ta")]
                                         ],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_TA',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[

In [15]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                             [("number", "d1"), ("shape", "ta")],
                                             [("number", "d2"), ("shape", "ta")],
                                             [("number", "d3"), ("shape", "ta")],
                                             [("number", "d5"), ("shape", "ta")]
                                             ],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_TA_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[

### Stimulus and 2 blanks before and 4 after

In [16]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                             [("number", "d1"), ("shape", "ta")],
                                             [("number", "d2"), ("shape", "ta")],
                                             [("number", "d3"), ("shape", "ta")],
                                             [("number", "d5"), ("shape", "ta")]
                                             ],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_TA',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 

In [17]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                             [("number", "d1"), ("shape", "ta")],
                                             [("number", "d2"), ("shape", "ta")],
                                             [("number", "d3"), ("shape", "ta")],
                                             [("number", "d5"), ("shape", "ta")]
                                             ],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_TA_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 

## 3. TP : equivalent total perimeter
### Only stimulus

In [18]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("shape", "tp")],
                                         [("number", "d2"), ("shape", "tp")],
                                         [("number", "d3"), ("shape", "tp")],
                                         [("number", "d5"), ("shape", "tp")]
                                         ],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_TP',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[

In [19]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                             [("number", "d1"), ("shape", "tp")],
                                             [("number", "d2"), ("shape", "tp")],
                                             [("number", "d3"), ("shape", "tp")],
                                             [("number", "d5"), ("shape", "tp")]
                                             ],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_TP_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[

### Stimulus and 2 blanks before and 4 after

In [20]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                             [("number", "d1"), ("shape", "tp")],
                                             [("number", "d2"), ("shape", "tp")],
                                             [("number", "d3"), ("shape", "tp")],
                                             [("number", "d5"), ("shape", "tp")]
                                             ],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_TP',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 

In [21]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                             [("number", "d1"), ("shape", "tp")],
                                             [("number", "d2"), ("shape", "tp")],
                                             [("number", "d3"), ("shape", "tp")],
                                             [("number", "d5"), ("shape", "tp")]
                                             ],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_TP_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 

## 4. CH : equivalent convex hull
### Only stimulus

In [22]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("spread", "ch")],
                                         [("number", "d2"), ("spread", "ch")],
                                         [("number", "d3"), ("spread", "ch")],
                                         [("number", "d5"), ("spread", "ch")]
                                         ],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_CH',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[

In [23]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("spread", "ch")],
                                         [("number", "d2"), ("spread", "ch")],
                                         [("number", "d3"), ("spread", "ch")],
                                         [("number", "d5"), ("spread", "ch")]
                                         ],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_CH_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[

### Stimulus and 2 blanks before and 4 after

In [24]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("spread", "ch")],
                                         [("number", "d2"), ("spread", "ch")],
                                         [("number", "d3"), ("spread", "ch")],
                                         [("number", "d5"), ("spread", "ch")]
                                         ],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_CH',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 

In [25]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("spread", "ch")],
                                         [("number", "d2"), ("spread", "ch")],
                                         [("number", "d3"), ("spread", "ch")],
                                         [("number", "d5"), ("spread", "ch")]
                                         ],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_CH_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 

## 5. ID : equivalent inter distance
### Only stimulus

In [26]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("spread", "id")],
                                         [("number", "d2"), ("spread", "id")],
                                         [("number", "d3"), ("spread", "id")],
                                         [("number", "d5"), ("spread", "id")]
                                         ],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_ID',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[

In [27]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("spread", "id")],
                                         [("number", "d2"), ("spread", "id")],
                                         [("number", "d3"), ("spread", "id")],
                                         [("number", "d5"), ("spread", "id")]
                                         ],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_ID_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[16  3 39  9]
[

### Stimulus and 2 blanks before and 4 after

In [28]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("spread", "id")],
                                         [("number", "d2"), ("spread", "id")],
                                         [("number", "d3"), ("spread", "id")],
                                         [("number", "d5"), ("spread", "id")]
                                         ],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_ID',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 

In [29]:
report.make_covariate_reports("SvB_max", group_to_plot,
                           "number",
                            conditions = [
                                         [("number", "d1"), ("spread", "id")],
                                         [("number", "d2"), ("spread", "id")],
                                         [("number", "d3"), ("spread", "id")],
                                         [("number", "d5"), ("spread", "id")]
                                         ],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v5","sig2v3", "sig2v5", "sig3v5",
                                               "sig1vB", "sig3vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_ID_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 13]
[14 15 16 17 18 19 20  1  2  3  4  5  6  7 37 38 39 40 41 42 43  7  8  9
 10 11 12 